### Download the model weights from original Darket's release

# TODO:
1. remove hardcoded filenames from `train_aml.py` and `train.py`
2. cleanup `model_data` folder to reduce size
3. upload `*.weights` to model registry to reduce size for aml/staging

In [57]:
# setting the model variables
yolo3_keras_model = 'model_data/tiny_yolo_weights.h5' # need to pass this as parameter to train.py
yolo3_weights_filename = 'model_data/yolov3-tiny.weights' # remove this from model_data after the keras conversion
yolo3_config = 'model_data/yolov3-tiny.cfg'
model_url = 'https://pjreddie.com/media/files/yolov3-tiny.weights'

In [56]:
# download the model weights

import urllib.request
urllib.request.urlretrieve(model_url, yolo3_weights_filename)

('model_data/yolo_v3-tiny.weights',
 <http.client.HTTPMessage at 0x7f3e70105828>)

## Install dependent packages

In [1]:
!pip uninstall -y azureml-automl-runtime azureml-train-automl-runtime
!pip install -U keras==2.2.4 tensorflow==1.14.0 tensorflow-gpu==1.14.0 pillow matplotlib h5py==2.10.0 tensorboard azureml-sdk==1.13.0 onnxruntime==1.4.0 onnx==1.7.0 azureml-widgets azureml-tensorboard azureml-opendatasets azureml-mlflow azureml-defaults azureml-contrib-services azureml-contrib-interpret

!pip install -U git+git://github.com/microsoft/onnxconverter-common.git@3451bbffe61a2591a17f4d99a405b48e9ae8e395
!pip install -U git+git://github.com/onnx/keras-onnx.git@ff17787c393e2ce34d43185447d7354525f3ba87

Uninstalling azureml-automl-runtime-1.4.0.post1:
  Successfully uninstalled azureml-automl-runtime-1.4.0.post1
Uninstalling azureml-train-automl-runtime-1.4.0.post1:
  Successfully uninstalled azureml-train-automl-runtime-1.4.0.post1
Requirement already up-to-date: keras==2.2.4 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (2.2.4)
     |████████████████████████████████| 109.2MB 55.9MB/s eta 0:00:01
     |████████████████████████████████| 377.0MB 72.6MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 41.5MB/s eta 0:00:01
     |████████████████████████████████| 11.6MB 44.1MB/s eta 0:00:01
     |████████████████████████████████| 2.9MB 25.3MB/s eta 0:00:01
     |████████████████████████████████| 6.8MB 30.3MB/s eta 0:00:01
Requirement already up-to-date: onnxruntime==1.4.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.4.0)
Requirement already up-to-date: onnx==1.7.0 in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.7.0)
     |████████████

### Convert the Darket model to Keras

In [59]:
# execute the pre-built conversion script provided in the sample
!python3 ./convert.py $yolo3_config $yolo3_weights_filename $yolo3_keras_model

Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.


Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 16)



2020-09-17 20:30:44.333594: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-09-17 20:30:44.341910: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-09-17 20:30:44.360571: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-09-17 20:30:44.360645: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (wopaulicompute): /proc/driver/nvidia/version does not exist
2020-09-17 20:30:44.363428: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequenc

### Training data
This sample uses the [VOC Pascal dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/#devkit) referred to as _VOCDevkit_.

To generate your own annotation file and class names file.

- One row for one image;  
- Row format: `image_file_path box1 box2 ... boxN`;  
- Box format: `x_min,y_min,x_max,y_max,class_id` (no space).  
- For VOC dataset, try `python voc_annotation.py`  
Here is an example:

        ```
        path/to/img1.jpg 50,100,150,200,0 30,50,200,120,3
        path/to/img2.jpg 120,300,250,600,2
        ...
        ```

In [ ]:
# setup the folder for the training dataset

import os

dataset_folder = 'training_dataset'
dataset_filename = dataset_folder + '/VOCtrainval2007.tar'
optional_dataset_filename = dataset_folder + '/VOCtest2007.tar'

if not os.path.exists(dataset_folder):
    os.makedirs(dataset_folder)

In [ ]:
dataset_url = 'http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar'
urllib.request.urlretrieve(dataset_url, dataset_filename)

In [ ]:
optional_dataset_url = 'http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar'
urllib.request.urlretrieve(optional_dataset_url, optional_dataset_filename)

Unpack the dataset and prepare to upload to Datastore in AML.

_This step will take a few minutes._

In [ ]:
! tar xf $dataset_filename
! tar xf $optional_dataset_filename
! python ./voc_annotation.py

In [4]:
#Initialize Workspace 
from azureml.core import Workspace

## existing AML Workspace in config.json
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

def_blob_store = ws.get_default_datastore()

orttrainingdemo
aifxdemo
eastus2
15ae9cb6-95c1-483d-a0e3-b1a1a3b06324


#### OR create a new workspace with the following steps

```
from azureml.core import Workspace

### Change this cell from markdown to code and run this if you need to create a workspace 
### Update the values for your workspace below
ws=Workspace.create(subscription_id="<subscription-id goes here>",
                resource_group="<resource group goes here>",
                name="<name of the AML workspace>",
                location="<location>")
                
ws.write_config()
```

Upload the VOCdevkit to the workspace datastore

In [ ]:
def_blob_store.upload("VOCdevkit", target_path="/data/VOCdevkit")

### Train the Keras model.

In [ ]:
import os
import shutil
import glob

#set the project folder
PROJECT_FOLDER = "./aml/staging"
if os.path.exists(PROJECT_FOLDER):
    shutil.rmtree(PROJECT_FOLDER)

os.makedirs(PROJECT_FOLDER, exist_ok=True)

# copy all python scripts to project folder
files = glob.glob("*.py")
for f in files:
    shutil.copy(f, PROJECT_FOLDER)

# copy all config files to the project folder
files = glob.glob("*.cfg")
for f in files:
    shutil.copy(f, PROJECT_FOLDER)

# copy all text files to the project folder
files = glob.glob("*.txt")
for f in files:
    shutil.copy(f, PROJECT_FOLDER)


In [ ]:
! cp -rf model_data $PROJECT_FOLDER # copy the dataset under the project folder

!cp -rf yolo3 $PROJECT_FOLDER # copy the model files under the project folder

In [ ]:
from azureml.data.data_reference import DataReference
from azureml.core import Dataset

training_dataset = Dataset.File.from_files(path=(def_blob_store, '/data/VOCdevkit'))

Add the optional packages and setup the training environment

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE

cd = CondaDependencies.create(pip_packages=['keras==2.1.5', 'tensorflow==1.6.0', 'pillow', 'matplotlib', 'h5py', 'tensorboard'], conda_packages=['python=3.6.11'])
myenv = Environment("yolov3")
myenv.python.conda_dependencies = cd
myenv.python.conda_dependencies.add_pip_package("azureml-sdk")
myenv.python.conda_dependencies.add_channel("conda-forge")
myenv.docker.enabled = True
myenv.docker.base_image = DEFAULT_GPU_IMAGE

Setup the training compute

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
CLUSTER_NAME = "gpu-taining"

# Verify that cluster does not exist already
try:
    aml_cluster = AmlCompute(workspace=ws, name=CLUSTER_NAME)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    print("provisioning new compute target")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_NC6", max_nodes=8, vm_priority="lowpriority"
    )
    aml_cluster = ComputeTarget.create(ws, CLUSTER_NAME, compute_config)

aml_cluster.wait_for_completion(show_output=True)


Define the run config for the experiment

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(
    source_directory=PROJECT_FOLDER,
    script='train.py',
    arguments=["--data", training_dataset.as_named_input('input').as_mount()],
    )

src.run_config.framework = 'python'
src.run_config.target = aml_cluster.name

# Set environment
src.run_config.environment = myenv

Run the training experiment

In [ ]:
from azureml.core import Experiment

EXPERIMENT_NAME = "keras-yolo3"

experiment = Experiment(workspace=ws, name=EXPERIMENT_NAME)

run = experiment.submit(config=src)

In [ ]:
%%time

run.wait_for_completion(show_output=True)

In [ ]:
# register the model in the model registry

my_trained_model = 'tiny_yolov3'

from azureml.core import Model
model = Model(ws, my_trained_model)

# download the trainied model
model.download(target_dir='model_data', exist_ok=True)

### Convert the model to ONNX and register in the model registry

In [6]:
model_path="model_data/trained_weights_final_manash.h5"
test_image="VOCdevkit/VOC2007/JPEGImages/000001.jpg"
anchors_path="model_data/tiny_yolo_anchors.txt"
classes_path="model_data/voc_classes.txt"

!python ./convert2onnx.py \
    --model_path $model_path \
    --test_image $test_image \
    --anchors_path $anchors_path \
    --classes_path $classes_path \
    --model_file_name 'new_yolo.onnx' \
    --overwrite

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/

Test this ONNX Model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

image = Image.open("VOCdevkit/VOC2007/JPEGImages/000001.jpg")
image_score = Image.open("VOCdevkit/VOC2007/JPEGImages/000001_score.jpg")
fig, ax = plt.subplots(1,2)
ax[0].imshow(image)
ax[1].imshow(image_score)
ax[0].axis('off')
_ = ax[1].axis('off')

In [ ]:
# Register the ONNX model in the workspace registry
from azureml.core.model import Model
Model.register(model_path = model_onnx, 
                model_name = "yolo3.onnx", 
                workspace = ws,
                description="ONNX model converted from trained Keras/Tf")

# CLEANUP!!!

In [ ]:
shutil.rmtree(PROJECT_FOLDER)

## FINISHED. 
Transition to the ADO to review status of the pipeline.